In [3]:
import win32com.client
import time
import shutil
import os
import codecs
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pywinauto import application
from pywinauto import timings

In [4]:
# 0: 날짜(ulong)
# 1:시간(long) - hhmm
# 2:시가(long or float)
# 3:고가(long or float)
# 4:저가(long or float)
# 5:종가(long or float)
# 6:전일대비(long or float) - 주) 대비부호(37)과 반드시 같이 요청해야 함
# 8:거래량(ulong or ulonglong) 주) 정밀도 만원 단위
# 9:거래대금(ulonglong)
# 10:누적체결매도수량(ulong or ulonglong) - 호가비교방식 누적체결매도수량
# 11:누적체결매수수량(ulong or ulonglong) - 호가비교방식 누적체결매수수량
#  (주) 10, 11 필드는 분,틱 요청일 때만 제공
# 12:상장주식수(ulonglong)
# 13:시가총액(ulonglong)
# 14:외국인주문한도수량(ulong)
# 15:외국인주문가능수량(ulong)
# 16:외국인현보유수량(ulong)
# 17:외국인현보유비율(float)
# 18:수정주가일자(ulong) - YYYYMMDD
# 19:수정주가비율(float)
# 20:기관순매수(long)
# 21:기관누적순매수(long)
# 22:등락주선(long)
# 23:등락비율(float)
# 24:예탁금(ulonglong)
# 25:주식회전율(float)
# 26:거래성립률(float)
# 37:대비부호(char) - 수신값은 GetHeaderValue 8 대비부호와 동일

In [5]:
# In[ ]:


class CREON(object):
    """대신증권 크레온 API"""
    
    def __init__(self):
        # 연결 여부 체크
        self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = self.objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            try:
                os.system("/Users/juhy9/Documents/GitHub/System/Quant/src/sudo/sudo_creon_plus_login.bat")
                time.sleep(100)
            except:
                exit()
     
    def setMethod(self, code, char, from_yyyymmdd=None, to_yyyymmdd=None, count=None):
        """
        count는 보통 상식의 데이터 개수가 아니다.
        여기서는 한번 요청 시 가져와지는 데이터의 개수이다.
        한번 요청 시 최대 2856개 가능하다.
        
        원하는 데이터 개수가 있으면 to_yyyymmdd 로 가져온 다음에 잘라서 사용한다.
        하루에 분단위 데이터가 381개이다. (* 마지막 10분은 동시호가)
        
        """
        # object 구하기
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.objStockChart.SetInputValue(0, code)  # 종목코드
        
        if to_yyyymmdd:
            self.objStockChart.SetInputValue(1, ord('1'))  # 요청 구분 '1': 기간, '2': 개수
            self.objStockChart.SetInputValue(2, from_yyyymmdd)  # To 날짜
            self.objStockChart.SetInputValue(3, to_yyyymmdd)  # From 날짜
        elif count:
            self.objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
            self.objStockChart.SetInputValue(4, count)  # 조회 개수
        else: raise print("기간을 입력해주세요.")
        
        if char == "m":
            # 날짜, 시간,시가,고가,저가,종가,거래량
            self.colnames = "날짜, 시간, 시가, 고가, 저가, 종가, 거래량".split(", ")
            self.objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])
        else:
            # 날짜,시가,고가,저가,종가,거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수
            self.colnames = "날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수".split(", ")
            self.objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9, 12, 13, 16, 20])
            
        self.objStockChart.SetInputValue(6, ord(char))  # '차트 주기 - 분/틱
        self.objStockChart.SetInputValue(7, 1)  # 분틱차트 주기
        
        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        
        
        
        self.data = {i: [] for i in self.colnames}
        
    def checkRequest(self):
        
        self.objStockChart.BlockRequest()
        
        rqStatus = self.objStockChart.GetDibStatus()
        
        if rqStatus != 0: 
            
            return False
        
#         else:
#             print("통신상태 양호, 누적 개수 {}".format(len(self.data["date"])))
        
        self.count = self.objStockChart.GetHeaderValue(3)
        
        if self.count <= 1: 
            
            return False
        
        return int(self.count)
    
    def checkRemainTime(self):
        
        # 연속 요청 가능 여부 체크
        remainTime = self.objCpCybos.LimitRequestRemainTime / 1000.
        remainCount = self.objCpCybos.GetLimitRemainCount(1)  # 시세 제한
        
        if remainCount <= 0:
            print("15초당 60건으로 제한합니다.")
            time.sleep(remainTime)
            
    
    def getStockPriceMin(self):
        
        while 1:
        
            self.checkRemainTime()
            rows = self.checkRequest()

            if rows:

                for i in range(rows):
                    
                    for idx, col in enumerate(self.colnames):
                    
                        self.data[col].append(self.objStockChart.GetDataValue(idx, i))
            else:

                break
                
    
        return self.data

In [6]:
# In[ ]:

KRX10JO_ETF_LIST = pd.read_csv("/Users/juhy9/Documents/GitHub/Quant/data/KRX10JO_ETF_LIST.csv")
KRX10JO_ETF_LIST = KRX10JO_ETF_LIST.values.tolist()

In [8]:
for yyyy in "2017 2018 2019".split(" "):
    
    for mm in "1 2 3 4 5 6 7 8 9 10 11 12".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for name, code, no, cap in tqdm(KRX10JO_ETF_LIST):

            creon = CREON()

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):

                os.makedirs(savedir)

            today = datetime.now().strftime("%Y%m%d")

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()

            DataFrame = pd.DataFrame(getStockPrice)

            tolist = DataFrame.values.tolist()
            
            if len(tolist) > 0:

                with codecs.open(savefile, "w", encoding='utf8') as f:
                    msg = " ".join(creon.colnames)
                    f.write(msg)
                    f.write("\n")

                    for to in reversed(tolist):
                        msg = " ".join([str(i) for i in to])
                        f.write(msg)
                        f.write("\n")

2017 01


 15%|████████████                                                                     | 39/263 [00:02<00:47,  4.75it/s]

15초당 60건으로 제한합니다.


 38%|██████████████████████████████▍                                                  | 99/263 [00:17<00:09, 16.60it/s]

15초당 60건으로 제한합니다.


...

 66%|████████████████████████████████████████████████████▌                           | 173/263 [00:30<00:07, 12.23it/s]

15초당 60건으로 제한합니다.


 89%|███████████████████████████████████████████████████████████████████████▍        | 235/263 [00:45<00:02, 13.39it/s]

15초당 60건으로 제한합니다.


100%|████████████████████████████████████████████████████████████████████████████████| 263/263 [01:00<00:00,  4.36it/s]
